
# Neuroscience Dashboard (Sample)

This dashboard uses the **current package** chosen in `Start.ipynb`.  
For the starter, it loads the built‑in **sample** (CSV + NPZ) included with the site.


In [ ]:

import os, json
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as W

DATA_ROOT = "data_pkgs"
CURRENT_FILE = os.path.join(DATA_ROOT, "CURRENT.txt")

def current_package_path():
    if not os.path.isfile(CURRENT_FILE):
        raise RuntimeError("No package selected. Open Start.ipynb.")
    with open(CURRENT_FILE, "r") as f:
        pid = f.read().strip()
    if pid == "sample_local":
        # built-in sample lives under content/data
        return "content/data"
    p = os.path.join(DATA_ROOT, pid)
    if not os.path.isdir(p):
        raise RuntimeError("Package folder missing. Re-select in Start.ipynb.")
    return p

PKG = current_package_path()
print("Using package:", PKG)

# Load meta and arrays
meta = pd.read_csv(os.path.join(PKG, "meta.csv"))
tun = np.load(os.path.join(PKG, "tuning_curves.npz"))
psth = np.load(os.path.join(PKG, "psth.npz"))
angles = tun["angles"]
time   = psth["t"]

# Basic filter widgets
layer_w = W.SelectMultiple(options=sorted(meta["layer"].unique()), description="Layer", value=tuple(sorted(meta["layer"].unique())))
osi_w   = W.FloatRangeSlider(min=0, max=1, step=0.01, value=[0.2, 0.9], description="OSI range", continuous_update=False)
sel_w   = W.FloatRangeSlider(min=0, max=1.2, step=0.01, value=[0.5, 1.1], description="Sel range", continuous_update=False)
maxu_w  = W.IntSlider(min=1, max=20, step=1, value=5, description="#Units")

plot_type = W.ToggleButtons(options=["Tuning", "PSTH"], value="Tuning", description="Plot")
ui = W.VBox([layer_w, osi_w, sel_w, maxu_w, plot_type])

def apply_filter():
    q = meta[meta["layer"].isin(layer_w.value)]
    lo, hi = osi_w.value; q = q[(q["osi"] >= lo) & (q["osi"] <= hi)]
    lo2, hi2 = sel_w.value; q = q[(q["sel_index"] >= lo2) & (q["sel_index"] <= hi2)]
    return q.head(maxu_w.value)

def plot():
    q = apply_filter()
    display(q)  # show matched table
    if plot_type.value == "Tuning":
        plt.figure(figsize=(6,3))
        for uid in q["unit_id"]:
            y = tun[uid]
            plt.plot(angles, y, alpha=0.8, label=uid)
        plt.title(f"Tuning curves (n={len(q)})")
        plt.xlabel("Angle (deg)"); plt.ylabel("Resp (a.u.)")
        plt.tight_layout(); plt.show()
    else:
        plt.figure(figsize=(6,3))
        for uid in q["unit_id"]:
            y = psth[uid]
            plt.plot(time, y, alpha=0.8, label=uid)
        plt.title(f"PSTH (n={len(q)})")
        plt.xlabel("Time (s)"); plt.ylabel("Rate (Hz)")
        plt.tight_layout(); plt.show()

out = W.Output()
def refresh(_=None):
    out.clear_output(wait=True)
    with out: plot()

for w in (layer_w, osi_w, sel_w, maxu_w, plot_type):
    w.observe(refresh, names="value")

refresh()
W.HBox([ui, out])



### Notes
- Add more pages (new notebooks) for rasters, per-trial views, etc.
- For large real data, create **zip packages** (CSV/NPZ) on cloud storage and add them to the package list in `Start.ipynb`.
